# MRKL Chat

This notebook showcases using an agent to replicate the MRKL chain using an agent optimized for chat models.

This uses the example Chinook database.
To set it up follow the instructions on https://database.guide/2-sample-databases-sqlite/, placing the `.db` file in a notebooks folder at the root of this repository.

In [2]:
from langchain import OpenAI, LLMMathChain, SerpAPIWrapper, SQLDatabase, SQLDatabaseChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

In [3]:
llm = ChatOpenAI(temperature=0)
llm1 = OpenAI(temperature=0)
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm1, verbose=True)
db = SQLDatabase.from_uri("sqlite:///../../../../../notebooks/Chinook.db")
db_chain = SQLDatabaseChain(llm=llm1, database=db, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
    Tool(
        name="FooBar DB",
        func=db_chain.run,
        description="useful for when you need to answer questions about FooBar. Input should be in the form of a question containing full context"
    )
]

In [4]:
mrkl = initialize_agent(tools, llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [5]:
print(mrkl.agent.llm_chain.prompt.messages[0].prompt.template)

Answer the following questions as best you can. You have access to the following tools:

> Search: useful for when you need to answer questions about current events. You should ask targeted questions
Args: {{'tool_input': {{'type': 'string'}}}}
> Calculator: useful for when you need to answer questions about math
Args: {{'tool_input': {{'type': 'string'}}}}
> FooBar DB: useful for when you need to answer questions about FooBar. Input should be in the form of a question containing full context
Args: {{'tool_input': {{'type': 'string'}}}}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are: Search, Calculator, FooBar DB

The $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:

```
{{
  "action": $T

In [6]:
mrkl.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")



> Entering new AgentExecutor chain...
Question: Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?
Thought: I need to use two different tools for this question. First, I need to use Search to find out who Leo DiCaprio's girlfriend is. Then, I need to use Calculator to calculate her current age raised to the 0.43 power.
Action:
```
{
  "action": "Search",
  "action_input": {
      "query": "Who is Leo DiCaprio's girlfriend?"
  }
}
```

Observation: Leonardo DiCaprio has split from girlfriend Camila Morrone. Getty. The Titanic actor hasn't been in a relationship with a woman over the age of ...
Thought:I need to use Search again to find out Camila Morrone's current age.
Action:
```
{
  "action": "Search",
  "action_input": {
      "query": "Camila Morrone age"
  }
}
```


Observation: 25 years
Thought:Now I need to use Calculator to calculate her current age raised to the 0.43 power.
Action:
```
{
  "action": "Calculator",
  "action_input": {
      "exp

'3.991298452658078'

In [7]:
mrkl.run("What is the full name of the artist who recently released an album called 'The Storm Before the Calm' and are they in the FooBar database? If so, what albums of theirs are in the FooBar database?")



> Entering new AgentExecutor chain...
Thought: I should use the Search tool to find the name of the artist who recently released an album called 'The Storm Before the Calm'
Action:
```
{
  "action": "Search",
  "action_input": {
      "query": "artist who recently released album 'The Storm Before the Calm'"
  }
}
```

Observation: Alanis Morissette
Thought:I should use the FooBar DB tool to check if Alanis Morissette is in the database and what albums of hers are in it.
Action:
```
{
  "action": "FooBar DB",
  "action_input": {
      "question": "What albums does Alanis Morissette have in the database?"
  }
}
```


> Entering new SQLDatabaseChain chain...
What albums does Alanis Morissette have in the database?
SQLQuery:

/Users/harrisonchase/.pyenv/versions/3.9.1/envs/langchain/lib/python3.9/site-packages/sqlalchemy/sql/sqltypes.py:726: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  util.warn(


 SELECT "Title" FROM "Album" WHERE "ArtistId" IN (SELECT "ArtistId" FROM "Artist" WHERE "Name" = 'Alanis Morissette') LIMIT 5;
SQLResult: [('Jagged Little Pill',)]
Answer: Alanis Morissette has the album Jagged Little Pill in the database.
> Finished chain.

Observation:  Alanis Morissette has the album Jagged Little Pill in the database.
Thought:I have found that Alanis Morissette is the artist who recently released an album called 'The Storm Before the Calm' and that she has the album Jagged Little Pill in the FooBar database.
Final Answer: Alanis Morissette is the artist who recently released an album called 'The Storm Before the Calm' and she has the album Jagged Little Pill in the FooBar database.

> Finished chain.


"Alanis Morissette is the artist who recently released an album called 'The Storm Before the Calm' and she has the album Jagged Little Pill in the FooBar database."